# Setting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import glob

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
!pip install category_encoders
import category_encoders as ce

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 874.8 kB/s eta 0:00:00


In [3]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/test.csv')
df_holidays = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/holidays_in_japan.csv')
df_venue = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/venue_information.csv')
df_match_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/match_reports.csv')

In [4]:
# kfインスタンスを作る(KFoldの設定)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [5]:
df_train_x = df_train.drop(['id', 'attendance'], axis=1)
df_train_y = df_train['attendance']
df_test_x = df_test.drop(['id'], axis=1)

# Feature engineering

In [6]:
# 日付をdatetime型にする
df_holidays['holiday_date'] = pd.to_datetime(df_holidays['holiday_date'])
df_train_x['match_date'] = pd.to_datetime(df_train_x['match_date'])
df_test_x['match_date'] = pd.to_datetime(df_test_x['match_date'])

In [7]:
# 休日なら1、平日なら0のholidays列をデータフレームに追加
df_train_x['holidays'] =df_train_x['match_date'].dt.dayofweek.apply(lambda x: 1 if(x in df_holidays['holiday_date'].values or x in [5,6]) else 0)
df_test_x['holidays'] =df_test_x['match_date'].dt.dayofweek.apply(lambda x: 1 if(x in df_holidays['holiday_date'].values or x in [5,6]) else 0)

In [8]:
# sectionとroundからsection_roundを作る
df_train_x['section_round'] = df_train_x['section'] + '_' + df_train_x['round']
df_test_x['section_round'] = df_test_x['section'] + '_' + df_test_x['round']
df_train_x = df_train_x.drop(['section', 'round'], axis=1)
df_test_x = df_test_x.drop(['section', 'round'], axis=1)

In [9]:
# kick_off_timeをhourだけにする
df_train_x['kick_off_time'] = pd.to_datetime(df_train_x['kick_off_time'], format='%H:%M').dt.hour
df_test_x['kick_off_time'] = pd.to_datetime(df_test_x['kick_off_time'], format='%H:%M').dt.hour

In [10]:
# 都道府県を見つける関数
def extract_prefecture(address):
  if '県' in address:
    return address[:address.find('県')+1]
  elif '道' in address:
    return address[:address.find('道')+1]
  elif '府' in address:
    return address[:address.find('府')+1]
  elif '都' in address:
    return address[:address.find('都')+1]
  else:
    return None

In [11]:
# df_venueに都道府県を示すprefecture列を追加
df_venue['prefecture'] = df_venue['address'].apply(extract_prefecture)
df_venue.head()

,venue,capacity,address,prefecture
0,札幌ドーム,38794,北海道札幌市豊平区羊ヶ丘1,北海道
1,埼玉スタジアム2002,62010,埼玉県さいたま市緑区中野田500,埼玉県
2,味の素スタジアム,47851,東京都調布市西町376−3,東京都
3,町田GIONスタジアム,15320,東京都町田市野津田町2035,東京都
4,日産スタジアム,71624,神奈川県横浜市港北区小机町3300,神奈川県


In [12]:
# データにvenueをマージして、capacity列,prefecture列を追加
df_train_x = df_train_x.merge(df_venue[['venue', 'prefecture', 'capacity']], on='venue', how='left')
df_test_x = df_test_x.merge(df_venue[['venue', 'prefecture', 'capacity']], on='venue', how='left')

In [13]:
# obj型とそれ以外を分ける
df_train_x_obj = df_train_x.select_dtypes(include='object')
df_test_x_obj = df_test_x.select_dtypes(include='object')
df_train_x_num = df_train_x.select_dtypes(exclude='object')
df_test_x_num = df_test_x.select_dtypes(exclude='object')

In [14]:
# objをエンコーダーで数字を割り振る
list_cols = df_train_x_obj.columns
encoder = ce.OrdinalEncoder(cols=list_cols)
df_train_x_obj_enc = encoder.fit_transform(df_train_x_obj)
df_test_x_obj_enc = encoder.transform(df_test_x_obj)

In [15]:
df_train_x_coc = pd.concat([df_train_x_num, df_train_x_obj_enc], axis=1)
df_test_x_coc = pd.concat([df_test_x_num, df_test_x_obj_enc], axis=1)

In [16]:
# 何月かを示すmatch_monthを作成
df_train_x_coc['match_month'] = df_train_x_coc['match_date'].dt.month
df_test_x_coc['match_month'] = df_test_x_coc['match_date'].dt.month
df_train_x_coc.head()

,match_date,kick_off_time,temperature,humidity,holidays,capacity,home_team,away_team,venue,weather,broadcasters,section_round,prefecture,match_month
0,2006-03-04,16,8.3,40,1,21000,1,1,1,1,1,1,1,3
1,2006-03-05,13,12.9,28,1,15859,2,2,2,1,2,2,2,3
2,2006-03-05,13,12.1,35,1,47851,3,3,3,1,3,2,3,3
3,2006-03-05,14,11.6,42,1,51697,4,4,4,1,4,2,4,3
4,2006-03-05,14,13.1,32,1,20223,5,5,5,1,5,2,5,3


In [17]:
df_train_x_coc = df_train_x_coc.drop(['match_date'], axis=1)
df_test_x_coc = df_test_x_coc.drop(['match_date'], axis=1)

# Prediction

In [18]:
# 評価用と訓練用にデータを分ける
x_train_pra, x_train_val, y_train_pra, y_train_val = train_test_split(df_train_x_coc, df_train_y, test_size=0.2)

In [19]:
reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg.fit(x_train_pra, y_train_pra)
y_pred = reg.predict(x_train_val)
print(mean_squared_error(y_train_val, y_pred))

19654099.64265592


In [20]:
test_pred = reg.predict(df_test_x_coc)

In [22]:
submission = pd.DataFrame({'id': df_test['id'], 'attendance': test_pred})
submission.to_csv('Deroito_J_league_12.csv', index=False, header=False)